In [16]:
import json
import os

from openai import AzureOpenAI
from azure.core.credentials import AzureKeyCredential

from azure.identity import DefaultAzureCredential
from sqlalchemy import create_engine
from sqlalchemy import Index, create_engine, select, text
from sqlalchemy.orm import DeclarativeBase, Mapped, Session, mapped_column
import urllib

from dotenv import load_dotenv
import os

load_dotenv(override=True)

True

In [17]:
os.getenv("AZURE_ASSISTANT_ID")

'asst_Oq6t2LdLdnDIl8HJH6QXdTmq'

In [18]:
def query_db(query: str) -> dict:        
        print("Authenticating to Azure Database for PostgreSQL using Azure Identity...")
        POSTGRES_HOST = os.environ["POSTGRES_HOST"]
        POSTGRES_USERNAME = os.environ["POSTGRES_USERNAME"]
        POSTGRES_DATABASE = os.environ["POSTGRES_DATABASE"]
        POSTGRES_SSL = os.environ.get("POSTGRES_SSL")

        azure_credential = DefaultAzureCredential()
        token = azure_credential.get_token("https://ossrdbms-aad.database.windows.net/.default")
        POSTGRES_PASSWORD = token.token

        DATABASE_URI = f"postgresql://{POSTGRES_USERNAME}:{POSTGRES_PASSWORD}@{POSTGRES_HOST}/{POSTGRES_DATABASE}"
        # Specify SSL mode if needed
        DATABASE_URI += f"?sslmode={POSTGRES_SSL}"

        engine = create_engine(DATABASE_URI, echo=False)

        # Run query
        results = []
        with Session(engine) as session:
                most_similars = session.execute(text(query)) #.scalars()
                #print(f"Most similar recipes to 'chicken and peanut':")
                #print(f"--------------------------------------------------")
                for Recipe in most_similars:
                        results.append({"recipe": Recipe[1], "score": Recipe[2]})
                        #print(f"INPUT: {Recipe[0]}: \nOUTPUT: {Recipe[1]} SCORE: ({Recipe[2]})")
                        #print(f"--------------------------------------------------")
        
        
        return json.dumps(results)

In [19]:
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from openai import AzureOpenAI

token_provider = get_bearer_token_provider(
    DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default"
)

# Initialize the Azure OpenAI client
client = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
    #api_key=os.getenv("AZURE_OPENAI_KEY"),  
    azure_ad_token_provider=token_provider,
    api_version="2025-02-01-preview"
)


reviewer_coder_system_message="""
    Your responsibility is to chat with the user and to make sure that the response is coherent with the user question.
    
    Follow these instructions to assist the user:
    STEPS:
    1. Review the user question and rephrase it if needed. Don't try to answer the question directly.
    2. Refer to the chat history to understand the context of the conversation.
    3. Given an input question, create a syntactically correct SQL query, and call the query_db function to execute the query and return the resulting data in markdown format.
    4. Format the data using markdown format.
    5. Review the data returned by the Reviewer agent and format it to return the appropriate response.
    6. If the data is not satisfactory, provide specific suggestions to the Coder agent to improve the data.
    7. If the data is satisfactory, return it to the user.

    The SQL query must be syntactically correct to run on a PostgreSQL database and should not contain any errors.
    The SQL query must be based on the following tables and fields:
    
    PostgreSQL "recommend_recipe_by_description" table has properties:
        #
        #  in_recipedescription character varying "input recipe description"
        #  out_recipename character varying "similar recipe name"
        #  out_similarityscore real "similarity score between 0 and 1"
        #  

            
    Examples:
    Question: What recipes are similar to 'chicken and peanut'?

    Query:
    SELECT in_recipedescription, out_recipename, out_similarityscore 
    FROM recommend_recipe_by_description('chicken and peanut', 3)
    WHERE out_similarityscore !=0
    ORDER BY 2 DESC
    ; 

    Answer:
    The top 3 most similar recipes to 'chicken and peanut' are in markdown format: 
    markdown table example:
    | Recipe Name | Similarity Score |
    | ------------- | ----------------- |
    | Recipe 1 | 0.95 |
    | Recipe 2 | 0.90 |
    | Recipe 3 | 0.85 |


    RULES:
    - Only identify suggestions that are specific and actionable.
    - Verify previous suggestions have been addressed.
    - Never try to simulate the data, always call the Coder agent to get the data.
    - Never repeat previous suggestions.
    - Never generate SQL query, let the coder agent do that.
    - **MUST** nerver make up data or use data from other tables. Use only result returned by the surveyDataInsightsPlugin function.
    """
 
assistant = client.beta.assistants.create(
    instructions=reviewer_coder_system_message,
    model="gpt-4o",
    tools=[{"type": "code_interpreter"},
            {
                "type": "function",
                "function": {
                    "name": "query_db",
                    "description": "Execute SQL query",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "query": {
                                "type": "string",
                                "description": "query to be executed in SQL database",
                            },
                        },
                        "required": ["query"],
                    },
                }
            }
    ]
    )

In [20]:
print("assistant")
assistant.to_dict()

assistant


{'id': 'asst_FtYhCpOr1clXR698M9rMI35h',
 'created_at': 1744749727,
 'description': None,
 'instructions': '\n    Your responsibility is to chat with the user and to make sure that the response is coherent with the user question.\n    \n    Follow these instructions to assist the user:\n    STEPS:\n    1. Review the user question and rephrase it if needed. Don\'t try to answer the question directly.\n    2. Refer to the chat history to understand the context of the conversation.\n    3. Given an input question, create a syntactically correct SQL query, and call the query_db function to execute the query and return the resulting data in markdown format.\n    4. Format the data using markdown format.\n    5. Review the data returned by the Reviewer agent and format it to return the appropriate response.\n    6. If the data is not satisfactory, provide specific suggestions to the Coder agent to improve the data.\n    7. If the data is satisfactory, return it to the user.\n\n    The SQL que

In [21]:
query_db("""SELECT in_recipedescription, out_recipename, out_similarityscore 
         FROM recommend_recipe_by_description('chicken and peanut', 3)
         WHERE out_similarityscore !=0\nORDER BY 2 DESC"""
         )



Authenticating to Azure Database for PostgreSQL using Azure Identity...


'[{"recipe": "Green Papaya Salad (Som Tam Malakor)", "score": 0.19567707}, {"recipe": "Chicken, Avocado and Mango Salad", "score": 0.19447339}, {"recipe": "Apricot Glazed Chicken", "score": 0.1933773}]'

In [22]:
thread = client.beta.threads.create()
message = client.beta.threads.messages.create(
  thread_id=thread.id,
  role="user",
  content="I am looking for recipe that contains chicken and peanut. Can you suggest me few similar recipes? Show me a graph",
)

In [23]:
from typing_extensions import override
from openai import AssistantEventHandler
from openai.types.beta.threads import ImageFile, Message, ImageFileDeltaBlock
import base64
import json
class EventHandler(AssistantEventHandler):
    @override
    def on_event(self, event):
      # Retrieve events that are denoted with 'requires_action'
      # since these will have our tool_calls
      if event.event == 'thread.run.requires_action':
        run_id = event.data.id  # Retrieve the run ID from the event data
        self.handle_requires_action(event.data, run_id)
 
    def handle_requires_action(self, data, run_id):
      tool_outputs = []
        
      for tool in data.required_action.submit_tool_outputs.tool_calls:
        if tool.function.name == "query_db":
          print(f"tool.function:{tool.function}")
          query = json.loads(tool.function.arguments)["query"]
          tool_outputs.append({"tool_call_id": tool.id, "output": query_db(query)})
        elif tool.function.name == "get_rain_probability":
          tool_outputs.append({"tool_call_id": tool.id, "output": "0.06"})
        """
        if tool.type == "code_interpreter":
          if tool.code_interpreter.input:
              tool_outputs.append(json.dumps(dict(code=tool.code_interpreter.input.split("\n"),
                                                            tool_call_id=tool.id)))

          if tool.code_interpreter.outputs:
              output_dict = {}
              for output in tool.code_interpreter.outputs:
                  if output.type == "logs":
                      output_dict["logs"] = output.logs.split("\n")
                  elif output.type == "image":
                      output_dict["image_file_id"] =  output.image.file_id
                      file_id = output.image.file_id
                      #image_base64 = Image(client.files.content(file_id).read()).to_base64(with_type=True)
                      image_base64 = base64.b64encode(client.files.content(file_id).read()).decode('utf-8')
                      output_dict["image_base64"] = image_base64
                      tool_outputs.append({"tool_call_id": tool.id, "output": image_base64})
          """
      
      # Submit all tool_outputs at the same time
      self.submit_tool_outputs(tool_outputs, run_id)
 
    def submit_tool_outputs(self, tool_outputs, run_id):
      # Use the submit_tool_outputs_stream helper
      with client.beta.threads.runs.submit_tool_outputs_stream(
        thread_id=self.current_run.thread_id,
        run_id=self.current_run.id,
        tool_outputs=tool_outputs,
        event_handler=EventHandler(),
      ) as stream:
        for text in stream.text_deltas:
          print(text, end="", flush=True)
        
        print()
        
 
with client.beta.threads.runs.stream(
  thread_id=thread.id,
  assistant_id=assistant.id,
  event_handler=EventHandler()
) as stream:
  stream.until_done()

tool.function:Function(arguments='{"query":"SELECT in_recipedescription, out_recipename, out_similarityscore FROM recommend_recipe_by_description(\'chicken and peanut\', 3) WHERE out_similarityscore !=0 ORDER BY 2 DESC;"}', name='query_db')
Authenticating to Azure Database for PostgreSQL using Azure Identity...
Here are the top 3 recipes that are most similar to "chicken and peanut":

| Recipe Name                      | Similarity Score |
|----------------------------------|------------------|
| Green Papaya Salad (Som Tam Malakor) | 0.1957           |
| Chicken, Avocado and Mango Salad  | 0.1945           |
| Apricot Glazed Chicken            | 0.1934           |

Now, I'll create a graph to visualize the similarity scores of these recipes.The graph above displays the similarity scores of recipes that are most similar to "chicken and peanut." Each bar represents a recipe, and the height indicates its similarity score. Let me know if you'd like further assistance!
